# Step 6 - Plotting bicycle network analysis results
## Project: Algorithmic bicycle network design
#### Michael Szell, Tyler Perlman, Sayat Mimar, Gourab Ghoshal, Roberta Sinatra

This notebook takes the results from 03_poi_based_generation and 04_connect_clusters and 05_analyze_results, and plots them.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-08  
Last modified: 2020-07-17

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

placeid = "vienna"

### Setup

In [ ]:
bbox = {}
bbox["inline"] = (250,250)
bbox["export"] = (800,800)

%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Existing infrastructure

In [ ]:
# Load networks
G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')

# Load POIs
with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
    nnids = [int(line.rstrip()) for line in f]

### Maps

In [ ]:
%%capture
my_plot_reset(G_carall, nnids)
ig.plot(G_carall, PATH["plots"] + placeid + "/" + placeid + '_carall_poi_' + poi_source + ".png", bbox = bbox["export"])
ig.plot(G_carall, bbox = bbox["inline"])

In [ ]:
%%capture
my_plot_reset(G_carall, [])
ig.plot(G_carall, PATH["plots"] + placeid + "/" + placeid + '_carall.png', bbox=bbox["export"])
ig.plot(G_carall, bbox=bbox["inline"])

In [ ]:
my_plot_reset(G_biketrack, [])
ig.plot(G_biketrack, PATH["plots"] + placeid + "/" + placeid + '_biketrack.png', bbox = bbox["export"])
ig.plot(G_biketrack, bbox = bbox["inline"])

In [ ]:
%%capture
my_plot_reset(G_bikeable, [])
ig.plot(G_bikeable, PATH["plots"] + placeid + "/" + placeid + '_bikeable.png', bbox = bbox["export"])
ig.plot(G_bikeable, bbox = bbox["inline"])

In [ ]:
%%capture
my_plot_reset(G_biketrackcarall, [])
ig.plot(G_biketrackcarall, PATH["plots"] + placeid + "/" + placeid + '_biketrackcarall.png', bbox = bbox["export"])
ig.plot(G_biketrackcarall, bbox=bbox["inline"])

#### Folium

In [ ]:
# maptiles = ('cartodbpositron','openstreetmap')
# edge_colors = ('#0033cc',)
# ox.folium.plot_graph_folium(G_biketrack, graph_map=None, 
#                                               popup_attribute=None, 
#                                               tiles=maptiles[0], 
#                                               zoom=19, 
#                                               fit_bounds=True, 
#                                               edge_color=edge_colors[0], 
#                                               edge_width=2, 
#                                               edge_opacity=.64)

### Analysis

## Generated, POI based

### Load data

In [ ]:
filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".pickle"
resultfile = open(PATH["results"] + placeid + "/" + filename,'rb')
res = pickle.load(resultfile)
resultfile.close()
if debug: pp.pprint(res)

### Maps

Abstract Greedy Triangulation

In [ ]:
for GT_abstract, prune_quantile in zip(res["GT_abstracts"], res["prune_quantiles"]):
    color_nodes(GT_abstract, "red", False, False)
    size_nodes(GT_abstract, 6, False, False)
    ig.plot(GT_abstract, PATH["plots"] + placeid + "/" + placeid + '_GTabstract_poi_'+ poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox = bbox["export"])

Just the bicycle network

In [ ]:
for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
    size_nodes(GT, 0)
    width_edges(GT, 2)
    color_edges(GT, "blue")
    ig.plot(GT, PATH["plots"] + placeid + "/" + placeid + '_GTbonly_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox=(800,800))

All together

In [ ]:
for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
    my_plot_reset(G_carall, nnids)
    color_nodes(G_carall, "red", nnids)
    size_nodes(G_carall, 8, nnids)
    GT_ids = [v["id"] for v in GT.vs]  
    color_nodes(G_carall, "blue", GT_ids)
    size_nodes(G_carall, 5, GT_ids)
    ig.plot(G_carall, PATH["plots"] + placeid + "/" + placeid + '_GTall_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox=(800,800))

### Analysis

## Generated, Cluster based

### Load data

In [ ]:
filename = placeid + '_clusters_' + prune_measure + "_cutoff" + cutofftype + "{:.2f}".format(cutoff) + ".pickle"
resultfile = open(PATH["results"] + placeid + "/" + filename,'rb')
res = pickle.load(resultfile)
resultfile.close()
if debug: pp.pprint(res)

### Maps

Just clusters

In [ ]:
plt.figure(figsize=[2*6.4, 2*4.8])
for i in range(len(res["clusters"])):
    plt.plot(res["clusters"][i].vs['x'],res["clusters"][i].vs['y'],'.')
plt.gca().invert_yaxis()
%config InlineBackend.figure_format = 'retina'

Abstract Greedy Triangulation

In [ ]:
for GT_abstract, prune_quantile in zip(res["GT_abstracts"], res["prune_quantiles"]):
    color_nodes(GT_abstract, "red", False, False)
    size_nodes(GT_abstract, 6, False, False)
    ig.plot(GT_abstract, PATH["plots"] + placeid + "/" + placeid + '_GTclustersabstract_biketrack_cutoff' + cutofftype + "{:.2f}".format(cutoff) + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox = bbox['export'])

Clusters with their new connections

In [ ]:
plt.figure(figsize=[2*6.4, 2*4.8])
plt.plot(res["GTs"][-1].vs['x'], res["GTs"][-1].vs['y'], 'o', color='red', markersize=5)
for i in range(len(res["clusters"])):
    plt.plot(res["clusters"][i].vs['x'], res["clusters"][i].vs['y'], 'o', markersize=3)
plt.gca().invert_yaxis()
%config InlineBackend.figure_format = 'retina'

Just the new connections

In [ ]:
size_nodes(res["GTs"][-1], 0)
width_edges(res["GTs"][-1], 2)
color_edges(res["GTs"][-1], "blue")
ig.plot(res["GTs"][-1])

The whole bike network and the new connections

In [ ]:
for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
    plt.figure(figsize=[2*6.4, 2*4.8])
    plt.plot(GT.vs['x'], GT.vs['y'], 'o', color='red', markersize=3)
    plt.plot(G_biketrack.vs['x'], G_biketrack.vs['y'], 'o', color='black', markersize=1)
    plt.gca().invert_yaxis()
    %config InlineBackend.figure_format = 'retina'
    plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_GTclusters_biketrack_cutoff' + cutofftype + "{:.2f}".format(cutoff) + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png')

### Analysis